### Question Answering Over Documents

Although LLMs are powerful, they do not know about information they were not trained on. If you want to use an LLM to answer questions about documents it was not trained on, you have to give it information about those documents. 

The idea is that for every question you want to ask chatGPT, you first do a retrieval step to fetch any relevant documents and then pass those documents, along with the original question, to the language model and have it generate a response. 


**Procedure:**

Prerequisites: 
- Install Libraries, Import libraries, set API key, and select models and tokenizers for embeddings.

Search (once per query)
- Given a user question, generate an embedding for the query from the OpenAI API
- Using the embeddings, rank the text sections by relevance to the query

Ask (once per query)
- Insert the question and the most relevant sections into a message to GPT
- Return GPT's answer

#### Quality Comparison
- This **notebook can** customize
    - GPT_MODEL
    - TEMP
    - PROMPT
    - MAX_TOKENS
    - KWARGS
    - MAX_LENGTH_OF_SUPPORTING_EVIDENCE

### 1. Prerequisites: 

Install Libraries, Import libraries, set API key, and select models and tokenizers for embeddings.

#### Install Libraries

```
% pip install --upgrade pip
% pip install git+https://github.com/openai/whisper.git
```

In [1]:
# imports
import os                  # for accessing files and Env Variables
import pandas as pd        # for storing text and embeddings data
import openai              # for calling the OpenAI API
import tiktoken            # for counting tokens

import llama_index                          # for crawling/scraping websites
from llama_index import download_loader     # for crawling/scraping websites

import numpy as np         # for updatind dataframes in numpy arrays for processing
from openai.embeddings_utils import distances_from_embeddings   # for embedding distance calcuation

# import json

#### Set API Keys

The OpenAI library will try to read your API key from the OPENAI_API_KEY environment variable. If you haven't already,find your [API Key](https://platform.openai.com/account/api-keys), and export your API key in your shell configuration file.

```
echo "export OPENAI_API_KEY='you-api-key'" >> ~/.zshrc
source ~/.zshrc
echo $OPENAI_API_KEY
```

In [2]:
# models, tokenizers & web domains

# Chunk: Documents are split into short, semi-self-contained sections to be embedded
TOKENIZER = "cl100k_base"                   # cl100k_base tokenizer is designed to work with the ada-002 model

# Embed: Each section is embedded with the OpenAI API
processed_data_dir = "../processed_data/"
EMBEDDING_MODEL = "text-embedding-ada-002"  # OpenAI's best embeddings as of Apr 2023
MAX_TOKENS = 1000                            # Max number of tokens in an embedded data Row, 750 tokens is ~1000 words.

# Store: Embeddings are saved in a CSV file
    
# Search (once per query)

MAX_LEN_OF_SUPPORTING_EVIDENCE = 2700       # The supporting evidence we send chatGPT with each Query

# Ask (once per query)
GPT_MODEL = "gpt-3.5-turbo"
TEST_QUERY = "Can you give me step by step intructions to integrated with Drift?"
TEMP=0.5
PROMPT="Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\""
URL_PREFIX = "https://www.evuniverse.com/"     #For URL of Supporting Evidence
DEBUG=False

### 2. Search (once per query)

This will take a user's question, create an embedding of it, compare it with the existing embeddings of our KB, retrieve the most relevant text based on the embedding value.

- Get the embedding value of the question
- Turn the KB embeddings into a NumPy array. This will provide more flexibility in how to use it given the many functions available that operate on NumPy arrays. It will also flatten the dimension to 1-D, which is the required format for many subsequent operations
- Identify Supporting Evidence to answer the question based on Embedding.  The search with embeddings compares the vector of numbers (which was the conversion of the raw text) using cosine distance. The vectors are likely related and might be the answer to the question if they are close in cosine distance. 


The text was broken up into smaller sets of tokens, so looping through in ascending order and continuing to add the text is a critical step to ensure a full answer. The max_len can also be modified to something smaller, if more content than desired is returned.

The previous step only retrieved chunks of texts that are semantically related to the question, so they might contain the answer, but there's no guarantee of it. The chance of finding an answer can be further increased by returning the top 5 most likely results.

In [3]:
# Identify Supporting Evidence to answer the question based on Embedding.  

def create_supporting_evidence(question, debug):
    
    # Turn the KB embeddings into a NumPy array.
    df=pd.read_csv(processed_data_dir + '/embeddings.csv', index_col=0)
    df['embeddings'] = df['embeddings'].apply(eval).apply(np.array)
    
    # Get the embeddings for the question
    q_embeddings = openai.Embedding.create(input=question, engine=EMBEDDING_MODEL)['data'][0]['embedding']
            
    # Identify Supporting Evidence to answer the question based on Embedding
    max_len=MAX_LEN_OF_SUPPORTING_EVIDENCE

    # Get the distances from the embeddings
    df['distances'] = distances_from_embeddings(q_embeddings, df['embeddings'].values, distance_metric='cosine')

    # print("Supporting Evidence Embedding Distances from the Query: ", df['distances'], "\n")
    
    source_texts = []
    source_urls = []
    cur_len = 0

    # Sort by distance and add the text to the context until the context is too long
    for i, row in df.sort_values('distances', ascending=True).iterrows():
        # Add the length of the text to the current length
        cur_len += row['n_tokens'] + 4

        # If the context is too long, break
        if cur_len > max_len:
            break

        # Else add it to the text that is being returned
        source_texts.append(row['text'])
        source_urls.append(URL_PREFIX + row['title'].strip(".txt"))


    # Update supporting evidence
    suporting_evidence = "\n\n###\n\n".join(source_texts)
    suporting_evidence_urls = "\n".join(source_urls)

    # Return the context
    return suporting_evidence, suporting_evidence_urls

#### 3. Ask (once per query)
- Insert the question and the most relevant sections into a message to GPT
- Answer the question based on the most similar supporting evidence

The answering prompt will then try to extract the relevant facts from the retrieved contexts, in order to formulate a coherent answer. If there is no relevant answer, the prompt will return “I don’t know”. A realistic sounding answer to the question can be created with the chat completion endpoint using gpt-3.5-turbo.

In [4]:
# Insert the question and the most relevant sections into a message to GPT

def answerMe(
    model=GPT_MODEL,
    question=TEST_QUERY,
    max_len=MAX_LEN_OF_SUPPORTING_EVIDENCE,
    debug=DEBUG,
    max_tokens=MAX_TOKENS,
    stop_sequence=None,
    temperature=TEMP,
):
    suporting_evidence, suporting_evidence_urls = create_supporting_evidence(question, debug)
     
    try:
        response = openai.ChatCompletion.create(
            temperature=temperature,
            model=model,
            top_p=1,
            stop=stop_sequence,
            max_tokens=max_tokens,
            frequency_penalty=0,
            presence_penalty=0,
            messages=[
                {"role": "system", "content": PROMPT + "\n\nsuporting_evidence: " + str(suporting_evidence) + "\n\n---\n\n"}, 
                {"role": "user", "content": str(question)}
            ]
        )     
        
        print(response.choices[0].message['content'], "\n")
        print("URLs For Further Reference:\n", suporting_evidence_urls, "\n\n")
        
        # If debug, print the raw model response
        if debug: 
            print("Question: ", TEST_QUERY, "\n")
            print("Answer: ", response.choices[0].message['content'], "\n\n")
            print("Prompt: ", PROMPT, "\n")        
            print("Raw Model Response:\n", response.choices[0].message, "\n")        
            print("Suporting Evidence:\n" + suporting_evidence)
            print("\n\n")            
    except Exception as e:
        print("\n\n Exception: ", e, "\n")
    return ""

#### Answer the question based on the most similar supporting evidence
It is done! A working Q/A system that has the knowledge embedded from EVUniverse Help Center is now ready. A few quick tests can be done to see the quality of the output:

In [6]:
answerMe(question="What is ChatGPT?")

I don't know. 

URLs For Further Reference:
 https://www.evuniverse.com/gmc-hummer-ev-review
https://www.evuniverse.com/ford-evs-gain-access-to-tesla-superchargers 




''

In [7]:
answerMe(question="How can I use EVUniverse?")

EVUniverse is a website that provides information and resources for people interested in electric vehicles (EVs). You can use it to learn about the costs, availability, and maintenance of EVs, as well as to explore comprehensive listings of EV models. Additionally, you can use EVUniverse to find public EV charging stations and learn how to use them. The website also provides articles and guides on various topics related to EVs, including charging, battery care, and software updates. 

URLs For Further Reference:
 https://www.evuniverse.com/are-public-ev-charge-stations-free
https://www.evuniverse.com/stay-charged-your-ultimate-guide-to-electric-car-maintenance 




''

In [8]:
answerMe(question="I am new to EVUniverse, what should I try next?")

Based on the context provided, you may want to try reading the article "Stay Charged: Your Ultimate Guide to Electric Car Maintenance" on EV Universe. It provides information on maintaining electric cars and is a good starting point for new EV owners. 

URLs For Further Reference:
 https://www.evuniverse.com/are-public-ev-charge-stations-free
https://www.evuniverse.com/stay-charged-your-ultimate-guide-to-electric-car-maintenance 




''

In [10]:
answerMe(question="What does electric car maintenance involve?")

Electric car maintenance involves regular checks and care needed to keep the electric vehicle running smoothly and efficiently. This includes tasks such as monitoring charge levels of the battery, checking tire pressure, rotating tires, checking the brake system, maintaining the cooling system, updating the car's software, and paying attention to the electric car charging equipment. While electric cars have fewer moving parts than their gasoline counterparts, they still require regular attention to ensure their high-tech components, particularly the battery and electric motor, continue to function optimally. 

URLs For Further Reference:
 https://www.evuniverse.com/stay-charged-your-ultimate-guide-to-electric-car-maintenance
https://www.evuniverse.com/are-public-ev-charge-stations-free 




''

In [11]:
answerMe(question="What kind of questions can I ask?")

You can ask any question related to the context provided, such as the cost of public EV charge stations, the availability of EV charge stations, the types of EV charging, the maintenance of electric cars, and more. 

URLs For Further Reference:
 https://www.evuniverse.com/are-public-ev-charge-stations-free
https://www.evuniverse.com/stay-charged-your-ultimate-guide-to-electric-car-maintenance 




''

In [9]:
answerMe(question="Create a Blog with 2000 words on how people would use EVUniverse", temperature=1)

As more and more people become conscious of the impact that fossil fuels are having on our planet, there is an increasing demand for clean, renewable energy sources that can power our daily lives. One of the most promising solutions to this problem is the electric vehicle (EV), which uses electric motors instead of internal combustion engines to power its wheels. But as with any new technology, there can be confusion and uncertainty about how to use it, which is why EV Universe is here to help. 

EV Universe is a comprehensive resource for all things electric vehicles, providing information on everything from the best EV models on the market to how to find and use public charging stations. With its intuitive interface and user-friendly approach, EV Universe is the perfect tool for anyone looking to make the switch to electric vehicles. In this blog post, we’ll explore how people can use EV Universe to make the transition to a cleaner, more sustainable future.

Choosing the Right EV Mod

''